<img src="figures/logos.png" style="float: centre;" width="800"/>

# Interseismic practical

## Written for the COMET InSAR Workshop 2025

Welcome to this COMET practical on interseismic strain accumulation.

This practical has been tested in the latest versions of Chrome and Firefox, but should work in most modern browsers.

### Interacting with this practical

To run a cell containing code, you can either press "Run" in the bar above, or press "Shift+Enter".

A "#" indicates a comment within the code. 

##################################### <br />
Blocks of code surrounded by multiple "#" (like this one) indicate variables that you can experiment with by changing the values. While you can experiment with the full body of code further (and I encourage you to do so after the course), we are limited in our ability to help you debug any errors you may encounter. Changing just the values should result in no major errors. <br />
#####################################

If you get a "dead kernel" error, either restart the kernel under "Kernel" on the top bar, or reopen the binder fresh. None of your changes here will affect the original binder, so feel free to reload it if anything breaks.

### Acknowledgements
Detailed guidance on the content was provided by [John Elliott](https://environment.leeds.ac.uk/see/staff/1248/dr-john-elliott). Advice on the computational requirements of the practical was provided by [Richard Rigby](https://environment.leeds.ac.uk/see/staff/2698/richard-rigby). This notebook was originally written by [Andrew Watson](https://www.linkedin.com/in/andrew-watson-a5248b1aa/?originalSubdomain=uk) using resources from the Tectonophysics ([Tim Wright](https://environment.leeds.ac.uk/see/staff/1613/professor-tim-wright)) and Inverse Theory ([Phil Livermore](https://environment.leeds.ac.uk/see/staff/1381/dr-phil-livermore)) undergraduate modules at the University of Leeds. The strain rate sections were originally written by [Qi Ou](https://environment.leeds.ac.uk/see/staff/11006/dr-qi-ou), with later modifications by [Jin Fang](https://environment.leeds.ac.uk/see/staff/13057/dr-jin-fang). Thanks to everyone for their contributions.

Presenter: [Jin Fang](https://environment.leeds.ac.uk/see/staff/13057/dr-jin-fang) (E-mail: J.Fang@leeds.ac.uk), University of Leeds. 


In [ ]:
# Import required modules. Run this before continuing.

# these are for performing the analysis
import numpy as np
from scipy import interpolate
from scipy import stats
from scipy import ndimage
import statistics
import pyproj
import subprocess as subp
from pathlib import Path
# this is our own library of functions, which can be found in the same directory as this notebook
import interseis_lib as lib

# this package is for general plotting
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

# this is additional scientific colour maps, see "https://www.fabiocrameri.ch/colourmaps/"
from cmcrameri import cm

# these packages are only needed for the final multivariate plot
import seaborn as sns
import pandas as pd

# this package is only needed for the strain rate calculation in the extension
import pygmt

# this refreshes the import of interseis_lib if changes are made while the notebook is running
import importlib
importlib.reload(lib)


## 0. Introduction

The earthquake cycle is a model for how earthquakes occur on faults. It consists of three phases:
- **Interseismic** - the fault is locked towards the surface but is accumulating strain at depth driven by external forces (i.e. plate motion).
- **Coseismic** - the strain is released seismically by slip on the previously-locked section of the fault.
- **Postseismic** - mechanisms after the earthquake, such as afterslip, poroelastic rebound, and/or viscoelastic relaxation.

We'll be focusing on the interseismic period for this practical. Figure 1 shows a schematic diagram of how the ground deforms through the interseismic period (middle panel) and into the coseismic (right panel). Profile A-A' shows a characteristic interseismic signal in the middle panel (the shape of an arctan function), where the ground displacement decreases from the far-field to zero at the fault trace.

<img src="figures/earthquakeCycle.png" style="float: centre;" width="600"/>

*Figure 1: Schematic map diagram of how expected ground deformation occurs through the interseismic and coseismic periods. From Elliott (2009).*

Geodesists are interested in this interseismic period as it provides information on the rate of strain accumulation and the size of the locked part of the fault, important parameters for understanding the seismic potential of a fault and the distribution of strain in the crust.

We can model interseismic deformation by considering steady slip on a screw dislocation embedded within an elastic half-space (Savage and Burford, 1973; Wright et al., 2013). In this simple model, the fault-parallel surface velocities, $v$, are a function of the perpendicular distance from the fault, $x$, the slip rate, $s$, and the locking depth, $d$:

\begin{equation}
\normalsize v(x) = \frac{s}{\pi}tan^{-1}\left(\frac{x}{d}\right) + c \qquad \qquad (1)
\end{equation}

Figure 2 shows a schematic diagram of this screw dislocation model.

<img src="figures/screwDisc.png" style="float: centre;" width="600"/>

*Figure 2: Schematic diagram of a screw dislocation model. Reproduced from Vernant (2015).*

Large volumes of InSAR data have been processed over the Xianshuihe Fault in southeastern Tibet by LiCS. We'll be using two of these LiCSBAS time series (026A_05924_131313 and 135D_06019_131313, shown in Figure 3) to estimate fault-parallel velocities, which we will then use to estimate the values of the parameters in Equation 1. LiCSBAS velocities for all these frames in SE Tibet are available on [Zenodo](https://doi.org/10.5281/zenodo.13731812). 

<img src="figures/XSHframeMap.png" style="float: centre;" width="800"/>

*Figure 3: Location of the two LiCS frames used in this practical, overlaid on faults (red lines) from [Styron (2022)](https://doi.org/10.1002/essoar.10512747.1). LiCSBAS velocities for all these frames are from [Fang et al. (2024)](https://doi.org/10.1029/2024GL111199), archived on [Zenodo](https://doi.org/10.5281/zenodo.13731812).*

This practical is broken down into the following sections:

**1)** Generating a simple synthetic screw dislocation model and investigating what we might expect InSAR to measure.

**2)** Loading in our LiCSBAS velocities and other data files.

**3)** Decomposing our two line-of-sight velocity maps into fault-parallel and vertical velocities, under the assumption of zero fault-perpendicular velocity. This includes resampling our velocities onto a unified grid and setting a common reference point.

**4)** Projecting the fault-parallel velocities onto a fault-perpendicular profile.

**5)** Performing a forward model to explore our profiled velocities.

**6)** Incorporation of surface fault creep in our model.

**7)** Estimating shear strain rate from 1-D velocity profiles

**8)** Estimating strain rates from 2-D surface velocities.

**9)** Extension - Performing a Bayesian inversion to estimate the parameters in Equation 1, along with their associated uncertainties.


For an example of this general method, have a look at our paper on the Main Recent Fault, Iran:
https://doi.org/10.1029/2021JB022674

## 1. Synthetic screw dislocation model

We'll start by generating some synthetic velocities using our screw dislocation model.

In [ ]:
import interseis_lib as lib

# set interseismic fault slip rate, s, and locking depth, d
#####################################
s = 20 # mm/yr          
d = 20 # km             
#####################################

# set up x coords (fault-perpendicular distances from -200 km to 200 km, with a sample every 2 km)
x = np.arange(-200, 202, 2)

# run the screw dislocation model. The 0 is just a static offset which we can ignore for now.
# We convert all units into SI (metres) so that they are consistent.
v = lib.screw_disc(x*1000, s/1000, d*1000, 0)

# expand into 2D, assuming our fault strikes East-West
v_grid = np.tile(v, (len(x),1)).T

# we'll also generate a model at s = 10 mm/yr and d = 10 km as a reference solution 
# so that you can see how changing s and d could affect the shape of the arctangent profile.
s_ref = 10 # mm/yr
d_ref = 10 # km
v_ref = lib.screw_disc(x*1000, s_ref/1000, d_ref*1000, 0)

# Plot the result
lib.plot_screw_disc(x, s, d, v, s_ref, d_ref, v_ref, v_grid)

The left plot shows our simple 1D screw dislocation model. The right shows this model expanded to 2D, assuming that our fault strikes from East to West. The fault-parallel velocities are also East-West velocities in this setup.

While these fault-parallel velocities are what we want for modelling a fault, they're not what we directly measure with InSAR. SAR satellites such as Sentinel-1 look down at the ground with an incidence angle, $\theta$, of around 31-46&deg;. This angle changes through our scene. InSAR is also only capable of measuring displacements towards and away from the satellite. The result of these two factors is that InSAR measures only a component of the observed deformation projected into the satellite's line-of-sight (LOS), and a single look direction cannot differentiate between different components of displacement.

<img src="figures/incAngle.png" style="float: centre;" width="300"/>

*Figure 4: Schematic diagram of satellite incidence angle, $\theta$, measured between the vertical (dashed black line) and the satellite line-of-sight (blue arrow).*

Let's apply a line-of-sight projection to our signals and see what effect this has on the velocities.

The interseismic_lib includes a function to make a grid of incidence angles between a minimum and maximum value, rotated by the satellite heading. Remember that the Sentinel-1 satellites are side-looking, so our incidence angles will change at 90&deg; to the satellite heading.

In [ ]:
# heading (-14 for ascending, -166 for descending), minimum incidence angle, and maximum incidence angle
#####################################
heading = -14 # degrees
inc_min, inc_max = 31, 46 # degrees
#####################################

# create grids of these angles
heading_grid = np.ones(v_grid.shape) * heading
inc_grid = lib.gen_inc(inc_min, inc_max, heading, x, x)

# projection from east-west to los
e2los = np.cos(np.deg2rad(heading_grid)) * np.sin(np.deg2rad(inc_grid))
v_grid_los = v_grid * e2los

# plot the 2D los and see how the incidence angle variation in range direction 
# changes the fault perpendicular velocity variation in the line of sight
lib.plot_screw_disc_in_los(x, v, inc_grid, v_grid, v_grid_los)

We can see that not only does the overall signal get smaller, but it also changes across the scene because of the varying incidence angle of the satellite.

Now that we've explored how a screw dislocation model might look in InSAR, we'll move on to applying the model to real-world data.

## 2. Load the input line-of-sight velocities

We'll begin by loading in our LiCSBAS outputs (you generated the equivalent outputs in the LiCSBAS practicals). These have been renamed to include the frame name for clarity. The _asc and _desc tags in the variable names indicate the ascending or descending frame.

**vel_file** - file containing the LOS velocities (found in TS_GEOCml10/results/vel).

**par_file** - parameter file which contains the geographic location of the data, along with the size in x and y (found in GEOCml##).

**E/N/U_file** - unit vector components, which basically tell us how to go from the satellite LOS to East, North, and Up. We need these to perform the velocity decomposition later.

In [ ]:
# setup file names
asc = '026A_05924_131313'
desc = '135D_06019_131313'

vel_file_asc = Path('data/' + asc + '.vel.filt.mskd')
par_file_asc = Path('data/' + asc + '.par')
E_file_asc = Path('data/' + asc + '.E')
N_file_asc = Path('data/' + asc + '.N')
U_file_asc = Path('data/' + asc + '.U')

vel_file_desc = Path('data/' + desc + '.vel.filt.mskd')
par_file_desc = Path('data/' + desc + '.par')
E_file_desc = Path('data/' + desc + '.E')
N_file_desc = Path('data/' + desc + '.N')
U_file_desc = Path('data/' + desc + '.U')

# read array dimensions from par file
width_asc = int(lib.get_par(par_file_asc,'width'))
length_asc = int(lib.get_par(par_file_asc,'nlines'))

width_desc = int(lib.get_par(par_file_desc,'width'))
length_desc = int(lib.get_par(par_file_desc,'nlines'))

# get corner positions
corner_lat_asc = float(lib.get_par(par_file_asc,'corner_lat'))
corner_lon_asc = float(lib.get_par(par_file_asc,'corner_lon'))

corner_lat_desc = float(lib.get_par(par_file_desc,'corner_lat'))
corner_lon_desc = float(lib.get_par(par_file_desc,'corner_lon'))

# get post spacing (distance between velocity measurements)
post_lat_asc = float(lib.get_par(par_file_asc,'post_lat'))
post_lon_asc = float(lib.get_par(par_file_asc,'post_lon'))

post_lat_desc = float(lib.get_par(par_file_desc,'post_lat'))
post_lon_desc = float(lib.get_par(par_file_desc,'post_lon'))

# calculate grid spacings
lat_asc = corner_lat_asc + post_lat_asc*np.arange(1,length_asc+1) - post_lat_asc/2
lon_asc = corner_lon_asc + post_lon_asc*np.arange(1,width_asc+1) - post_lon_asc/2

lat_desc = corner_lat_desc + post_lat_desc*np.arange(1,length_desc+1) - post_lat_desc/2
lon_desc = corner_lon_desc + post_lon_desc*np.arange(1,width_desc+1) - post_lon_desc/2

# load in velocities
vel_asc = np.fromfile(vel_file_asc, dtype='float32').reshape((length_asc, width_asc))
vel_desc = np.fromfile(vel_file_desc, dtype='float32').reshape((length_desc, width_desc))

# load in unit vectors
E_asc = np.fromfile(E_file_asc, dtype='float32').reshape((length_asc, width_asc))
N_asc = np.fromfile(N_file_asc, dtype='float32').reshape((length_asc, width_asc))
U_asc = np.fromfile(U_file_asc, dtype='float32').reshape((length_asc, width_asc))

E_desc = np.fromfile(E_file_desc, dtype='float32').reshape((length_desc, width_desc))
N_desc = np.fromfile(N_file_desc, dtype='float32').reshape((length_desc, width_desc))
U_desc = np.fromfile(U_file_desc, dtype='float32').reshape((length_desc, width_desc))

# load the frame polygons for plotting
poly_asc = np.loadtxt(Path('data/' + asc + '.poly.txt'))
poly_desc = np.loadtxt(Path('data/' + desc + '.poly.txt'))

# load the Xianshuihe fault trace
fault_trace = np.loadtxt(Path('data/XSH_trace.txt'))

print('Files loaded.')
print('Size of ascending velocity:')
print(vel_asc.shape)
print('Size of descending velocity:')
print(vel_desc.shape)


Next, we'll plot the velocities to make sure they've loaded in correctly. The trace of the Xianshuihe Fault will be plotted as a red line.

In [ ]:
# plot input vels
fig, axs = plt.subplots(1,2,figsize=(15,5))

# plot the ascending velocities, saturating the colour palette from -20 to 20 mm/yr.
im = axs[0].imshow(vel_asc, extent=[np.amin(lon_asc), np.amax(lon_asc), np.amin(lat_asc), np.amax(lat_asc)], \
                   cmap=cm.vik, vmin=-20, vmax=20)

# add the Xianshuihe fault trace
axs[0].plot(fault_trace[:,0], fault_trace[:,1], color="red")

# add the frame polygon
axs[0].plot(poly_asc[:,0], poly_asc[:,1], color="black", linestyle='dashed')

# display the colorbar, this "divider" section is to scale the colorbar size to the plot
divider = make_axes_locatable(axs[0])
cax = divider.append_axes("right", size="5%", pad="2%")
plt.colorbar(im, cax=cax, label='LOS velocity (mm/yr)')

# set the title
axs[0].set_title('026A_05924_131313')

# set the x and y limits of the plot
axs[0].set_xlim(np.amin(lon_asc), np.amax(lon_asc))
axs[0].set_ylim(np.amin(lat_asc), np.amax(lat_asc))

# label the x and y axes
axs[0].set_xlabel('Longitude (degrees)')
axs[0].set_ylabel('Latitude (degrees)')

# repeat for the descending velocities
im = axs[1].imshow(vel_desc, extent=[np.amin(lon_desc), np.amax(lon_desc), np.amin(lat_desc), np.amax(lat_desc)], \
                   cmap=cm.vik, vmin=-20, vmax=20)
axs[1].plot(fault_trace[:,0], fault_trace[:,1], color="red")
axs[1].plot(poly_desc[:,0], poly_desc[:,1], color="black", linestyle='dashed')
divider = make_axes_locatable(axs[1])
cax = divider.append_axes("right", size="5%", pad="2%")
plt.colorbar(im, cax=cax, label='LOS velocity (mm/yr)')
axs[1].set_title('135D_06019_131313')
axs[1].set_xlim(np.amin(lon_desc), np.amax(lon_desc))
axs[1].set_ylim(np.amin(lat_desc), np.amax(lat_desc))
axs[1].set_xlabel('Longitude (degrees)')
axs[1].set_ylabel('Latitude (degrees)')

# show the plot
fig.tight_layout()
plt.show()

By default, LiCSBAS sets motion towards the satellite as positive, and motion away from the satellite as negative. These velocities are relative to a LiCSBAS-selected reference point within each frame.

We can already see a change in the LOS velocities across the fault in both maps. To investigate further, let's take a profile across the fault in the ascending velocities.

## 3. Decompose the line-of-sight velocities into fault-parallel and vertical

Equation 1 requires fault-parallel velocities, while ours are currently in the satellite LOS.
We can estimate fault-parallel velocities by decomposing our LOS velocities into fault-parallel, fault-perpendicular, and vertical velocities.
This is done by solving a linear system of equations between our InSAR velocities ($V_{asc}$ and $V_{desc}$) and our component velocities.
We convert between the two using the radar incidence angle, $\theta$, and the azimuth of the along-track satellite heading, $\alpha$, and the strike of our fault, $\gamma$. Both $\theta$ and $\alpha$ vary pixel-by-pixel.

First, let's look at how we would decompose our LOS velocities into East ($V_E$), North ($V_N$), and vertical ($V_U$) velocities:
\begin{equation}
    \begin{bmatrix} V_{asc} \\ V_{desc} \end{bmatrix} = \begin{bmatrix} sin(\theta_{asc})cos(\alpha_{asc}) & sin(\theta_{asc})sin(\alpha_{asc}) & -cos(\theta_{asc}) \\ sin(\theta_{desc})cos(\alpha_{desc}) & sin(\theta_{desc})sin(\alpha_{desc}) & -cos(\theta_{desc}) \end{bmatrix} \begin{bmatrix} V_E \\ V_N \\ V_U \end{bmatrix}
    \qquad \qquad (2)
\end{equation}

We calculate fault-parallel ($V_{para}$) and fault-perpendicular ($V_{perp}$) velocities as a sum of relevant components of $V_E$ and $V_N$, including an extra term to account for the strike of the fault measured clockwise from North: 

\begin{equation}
    \begin{bmatrix} V_{asc} \\ V_{desc} \end{bmatrix} = \begin{bmatrix} \begin{pmatrix} sin(\gamma)sin(\theta_{asc})cos(\alpha_{asc})\\+ cos(\gamma)sin(\theta_{asc})sin(\alpha_{asc}) \end{pmatrix} & \begin{pmatrix} cos(\gamma)sin(\theta_{asc})cos(\alpha_{asc})\\+ sin(\gamma)sin(\theta_{asc})sin(\alpha_{asc}) \end{pmatrix} & -cos(\theta_{asc}) \\ \begin{pmatrix} sin(\gamma)sin(\theta_{desc})cos(\alpha_{desc})\\+ cos(\gamma)sin(\theta_{desc})sin(\alpha_{desc}) \end{pmatrix} & \begin{pmatrix} cos(\gamma)sin(\theta_{desc})cos(\alpha_{desc})\\+ sin(\gamma)sin(\theta_{desc})sin(\alpha_{desc}) \end{pmatrix} & -cos(\theta_{desc}) \end{bmatrix} \begin{bmatrix} V_{para} \\ V_{perp} \\ V_U \end{bmatrix}
    \qquad \qquad (3)
\end{equation}

In both cases, we have three unknowns and only two data sets, making this an underdetermined system.
Given that the amount of data available to us is fixed, we need to reduce the number of component velocities we want to estimate by one so that we can solve for the remaining two.
We'll assume that $V_{perp}$ is zero so that we can solve for $V_{para}$ and $V_U$. We choose $V_{perp}$ because we expect the dominant signal to be strike-slip motion on the Xianshuihe Fault.

We can now rewrite Equation 3:

\begin{equation}
    \begin{bmatrix} V_{asc} \\ V_{desc} \end{bmatrix} = \begin{bmatrix} sin(\gamma)sin(\theta_{asc})cos(\alpha_{asc})+cos(\gamma)sin(\theta_{asc})sin(\alpha_{asc}) & -cos(\theta_{asc}) \\ sin(\gamma)sin(\theta_{desc})cos(\alpha_{desc})+cos(\gamma)sin(\theta_{desc})sin(\alpha_{desc}) & -cos(\theta_{desc}) \end{bmatrix} \begin{bmatrix} V_{para} \\ V_U \end{bmatrix}
    \qquad \qquad (4)
\end{equation}

where $V_{asc}$ and $V_{desc}$ are the LOS velocities, $\theta$ is the radar incidence angle, and $\alpha$ is the azimuth of the along-track satellite heading. The conversion from LOS to fault-parallel can be simplified to $sin(\theta)sin(\alpha + \gamma)$, however, we only have the unit component vectors available, not the individual values of $\theta$ and $\alpha$, so we'll stick with Equation 4.

Currently, these velocities are on two slightly different grids. We need to unify the grids so that we can perform the velocity decomposition for shared points.

In [ ]:
# limits and intervals for new grid
lon_min = np.amin([lon_asc[0], lon_desc[0]])
lon_max = np.amax([lon_asc[-1], lon_desc[-1]])
lon_int = np.amin([post_lon_asc, post_lon_desc])
lon_regrid = np.arange(lon_min,lon_max+lon_int,lon_int)

lat_max = np.amax([lat_asc[0], lat_desc[0]])
lat_min = np.amin([lat_asc[-1], lat_desc[-1]])
lat_int = np.absolute(np.amin([post_lat_asc, post_lat_desc]))
lat_regrid = np.arange(lat_min,lat_max+lat_int,lat_int)

xx_regrid, yy_regrid = np.meshgrid(lon_regrid, lat_regrid[::-1])
coords_regrid = np.transpose(np.vstack((xx_regrid.flatten(),yy_regrid.flatten())))

# interpolate velocities onto this new grid
xx_asc, yy_asc = np.meshgrid(lon_asc, lat_asc)
vel_asc_regrid = interpolate.griddata((xx_asc.ravel(), yy_asc.ravel()), vel_asc.ravel(), (xx_regrid.ravel(), yy_regrid.ravel()))
vel_asc_regrid = vel_asc_regrid.reshape((len(lat_regrid),len(lon_regrid)))

xx_desc, yy_desc = np.meshgrid(lon_desc, lat_desc)
vel_desc_regrid = interpolate.griddata((xx_desc.ravel(), yy_desc.ravel()), vel_desc.ravel(), (xx_regrid.ravel(), yy_regrid.ravel()))
vel_desc_regrid = vel_desc_regrid.reshape((len(lat_regrid),len(lon_regrid)))

# interpolate the component vectors into the new grid
E_asc_regrid = interpolate.griddata((xx_asc.ravel(), yy_asc.ravel()), E_asc.ravel(), (xx_regrid.ravel(), yy_regrid.ravel())).reshape((len(lat_regrid),len(lon_regrid)))
N_asc_regrid = interpolate.griddata((xx_asc.ravel(), yy_asc.ravel()), N_asc.ravel(), (xx_regrid.ravel(), yy_regrid.ravel())).reshape((len(lat_regrid),len(lon_regrid)))
U_asc_regrid = interpolate.griddata((xx_asc.ravel(), yy_asc.ravel()), U_asc.ravel(), (xx_regrid.ravel(), yy_regrid.ravel())).reshape((len(lat_regrid),len(lon_regrid)))

E_desc_regrid = interpolate.griddata((xx_desc.ravel(), yy_desc.ravel()), E_desc.ravel(), (xx_regrid.ravel(), yy_regrid.ravel())).reshape((len(lat_regrid),len(lon_regrid)))
N_desc_regrid = interpolate.griddata((xx_desc.ravel(), yy_desc.ravel()), N_desc.ravel(), (xx_regrid.ravel(), yy_regrid.ravel())).reshape((len(lat_regrid),len(lon_regrid)))
U_desc_regrid = interpolate.griddata((xx_desc.ravel(), yy_desc.ravel()), U_desc.ravel(), (xx_regrid.ravel(), yy_regrid.ravel())).reshape((len(lat_regrid),len(lon_regrid)))

print('Regridding complete.')
print('Size of ascending velocity:')
print(vel_asc_regrid.shape)
print('Size of descending velocity:')
print(vel_desc_regrid.shape)

Currently, the two velocity fields are relative to their own reference pixels. We need to make this reference the same so that the velocities can be decomposed correctly. We'll define a reference window and use the mean of all the velocities within that area as the reference.

In [ ]:
# define new reference pixel
ref_xmin, ref_xmax = 100.4, 100.6
ref_ymin, ref_ymax = 29.45, 29.55

# ref poly for plotting
ref_poly = np.array([[ref_xmin, ref_ymin],
           [ref_xmin, ref_ymax],
           [ref_xmax, ref_ymax],
           [ref_xmax, ref_ymin],
           [ref_xmin, ref_ymin]])

# get index
ind_xmin = np.argmin(np.absolute(lon_regrid-ref_xmin))
ind_xmax = np.argmin(np.absolute(lon_regrid-ref_xmax))
ind_ymin = np.argmin(np.absolute(lat_regrid-ref_ymin))
ind_ymax = np.argmin(np.absolute(lat_regrid-ref_ymax))

# get ref value
ref_val_asc = np.nanmean(vel_asc_regrid[ind_ymin:ind_ymax+1,ind_xmin:ind_xmax+1])
ref_val_desc = np.nanmean(vel_desc_regrid[ind_ymin:ind_ymax+1,ind_xmin:ind_xmax+1])

# set as new reference
vel_asc_regrid = vel_asc_regrid - ref_val_asc
vel_desc_regrid = vel_desc_regrid - ref_val_desc

print('Referencing complete.')
print('Mean value of reference for ascending = ' + str(round(ref_val_asc,3)) + ' mm/yr')
print('Mean value of reference for descending = ' + str(round(ref_val_desc,3)) + ' mm/yr')

# plot the los velocity maps with our defined reference window
lib.plot_maps_with_reference(vel_asc_regrid, lon_regrid, lat_regrid, fault_trace, ref_poly, vel_desc_regrid)

Now we'll run the decomposition for each pixel, assuming no correlation between pixels and a strike of 130&deg;. 

In [ ]:
# set strike
strike = 130
strike_E, strike_N = np.sin(np.deg2rad(strike)), np.cos(np.deg2rad(strike))

# pre-allocate
vel_para = np.zeros((len(lat_regrid), len(lon_regrid)))
vel_U = np.zeros((len(lat_regrid), len(lon_regrid)))

# loop through every pixel
for ii in np.arange(0,len(lat_regrid)):
    for jj in np.arange(0,len(lon_regrid)):
        
        # create the design matrix
        G = np.array([[strike_E*E_asc_regrid[ii,jj] + strike_N*N_asc_regrid[ii,jj], U_asc_regrid[ii,jj]], 
                      [strike_E*E_desc_regrid[ii,jj] + strike_N*N_desc_regrid[ii,jj], U_desc_regrid[ii,jj]]])
        
        # get the two velocities for this pixel
        d = np.array([[vel_asc_regrid[ii,jj], vel_desc_regrid[ii,jj]]]).T
        
        # solve the linear system for the Up and East velocities
        m = np.linalg.solve(G, d)
        
        # save to arrays
        vel_para[ii,jj] = m[0, 0]
        vel_U[ii,jj] = m[1, 0]

# plot decomposed maps
lib.plot_decomposed_maps(vel_para, vel_U,lon_regrid, lat_regrid, fault_trace, poly_asc, poly_desc)


Note how we only get the decomposed velocities in the overlap area, where we have both line-of-sights to work with.

## 4. Profile the fault-parallel velocities

We'll now profile our decomposed fault-parallel velocities to produce a dataset which we can then apply our screw dislocation model to. First, we need to project our lat-long coordinates into Universal Transverse Mercator (UTM) so that the profile distances are in metres and not degrees. After this projection, our velocities will no longer be on a regular grid, so we'll plot them using scatter instead of imshow.

In [ ]:
# Convert to UTM using gdal

# get the utm zone projection
utm_crs_list = pyproj.database.query_utm_crs_info(
    datum_name="WGS 84",
    area_of_interest = pyproj.aoi.AreaOfInterest(
        west_lon_degree  = np.amin(lon_regrid),
        south_lat_degree = np.amin(lat_regrid),
        east_lon_degree  = np.amax(lon_regrid),
        north_lat_degree = np.amax(lat_regrid),
    ),
)
utm_crs = pyproj.CRS.from_epsg(utm_crs_list[0].code)

# create transformer for LL to UTM
transformer = pyproj.Transformer.from_crs('epsg:4326', utm_crs)

# apply transform to our grids of lat long coordinates
xx_utm, yy_utm = transformer.transform(yy_regrid, xx_regrid)

# apply transform to fault trace
fault_trace_utm = fault_trace.copy()
fault_trace_utm[:,0], fault_trace_utm[:,1] = transformer.transform(fault_trace[:,1], fault_trace[:,0])

# convert utm metres to km (easier to work with for plotting)
xx_utm = xx_utm / 1000
yy_utm = yy_utm / 1000
fault_trace_utm = fault_trace_utm / 1000

# get new limits for plotting
val_ind_row, val_ind_col = np.where(~np.isnan(vel_U).all(axis=0))[0], np.where(~np.isnan(vel_U).all(axis=1))[0]
xlim = [xx_utm[-1,val_ind_row[0]], xx_utm[-1,val_ind_row[-1]]]
ylim = [yy_utm[val_ind_col[-1],0], yy_utm[val_ind_col[0],0]]

# plot the utm velocity maps
lib.plot_utm_maps(xx_utm, yy_utm, vel_para, vel_U, fault_trace_utm, xlim, ylim)

In [ ]:
# Set the profile parameters

# rough middle profile
#####################################
prof_start = (830, 3410)
prof_end = (650, 3265)
prof_params = {
    "nbins": 100, # number of bins to split the profile into
    "width": 5  # total width of the profile in km
}
#####################################

# run the profiler, the outputs are as follows:
# - bin_val = mean value of each bin
# - prof_bin_mids = distance along the profile to the middle of each bin
# - points_val = every velocity within the profile
# - points_dist = distance along the profile to every point within the profile
# - points_poly = polygon that defines the profile
bin_val, prof_bin_mids, points_val, points_dist, points_poly \
    = lib.profile_data(xx_utm,yy_utm,vel_para,prof_start,prof_end,prof_params)

# calculate profile-fault intersection angle
intersect_dist, intersect_angle = lib.profile_fault_intersection(prof_start,prof_end,fault_trace_utm)

# plot the profile in map view and in cross-section
lib.plot_profile(xx_utm, yy_utm, vel_para, fault_trace_utm, prof_start, prof_end, points_poly, xlim, ylim, 
                 points_dist, points_val, prof_bin_mids, bin_val, intersect_dist, intersect_angle) 

I've included the intersection angle between the fault and the profile line on the right-hand plot.
We want this angle to be 90&deg;, otherwise the signal will become smeared and affect our results.

The pink dot marks the start of the profile, and the black dot marks the end. The three red lines between these show the main profile line and the extent of the profile width.

Our profiled velocities are shown on the right. These are projected onto the profile line, and so are given at distance along the profile from prof_start, which is located at 0 distance. The units of distance are the same as those for the profiled data set. The vertical dashed line marks where the profile line intersects the Xianshuihe fault trace.

While the profiled velocities are a little noisy, you should be able to see an overall change in velocity across the fault with a shape that resembles an arctangent function.

## 5. Forward model

Now that we have our data, let's produce a simple forward model to get an idea of what model parameters produce a reasonable fit.

In [ ]:
# Slip, locking depth, and offset
#####################################
s = 10 # mm/yr
d = 10 # km
c = -5 # mm/yr
#####################################

# shift profile so that the fault is at 0 km
x_prof = points_dist - intersect_dist

# Vector of positions every 1 km for -120 km to 120 km
x = np.arange(-120,121,1)

# run the forward model
v = lib.screw_disc(x*1000, s/1000, d*1000, c/1000)

# calculate rms misfit by running the forward model for x positions with velocities
v_forward = lib.screw_disc(x_prof*1000, s/1000, d*1000, c/1000)
rms_forward = lib.rms_misfit(points_val,v_forward*1000)

# Plot comparison
lib.plot_profile_model(x_prof, points_val, x, v, s, d, c, rms_forward)


To see how well you can all model the fault, let's model the same profile and then share the results.

Once you have your best model, please share your best-fit parameters in this form:

https://forms.gle/85DmMvrSPBmhmNY46

## 6. Fault creep

Depending on where you have drawn your profile lines so far, you may have noticed a sharp step in the velocities across the fault that is larger than the noise level. This may be fault creep, where the fault is also slipping aseismically at a shallow depth up to the surface. This fault creep occurs alongside the deeper interseismic slip.

A schematic diagram of fault creep is shown in Figure 5.

<img src="figures/faultCreep.png" style="float: centre;" width="500"/>

*Figure 5: (a) Schematic diagram of our fault creep model. (b) Forward model showing how the fault creep is expected to change our observed signal. From Hussian et al. (2016).*

We can model a combination of interseismic strain accumulation and fault creep to calculate a fault parallel velocity, $v$, at fault-perpendicular distance, $x$, using Equation 5:

\begin{equation}
v(x) = -\frac{s}{\pi}arctan\left(\frac{x+x_c}{d_1}\right) + C\left[\frac{1}{\pi}arctan\left(\frac{x+x_c}{d_2}\right) - H(x+x_c)\right] + c \qquad \qquad (5)
\end{equation}

We retain the screw dislocation model from before, where slip, $s$ occurs below a depth, $d_1$. For the shallow creep, occurring at a rate of $C$ between the surface and depth $d_2$, we use a back slip approach. This models the creep as the sum of slip on the entire fault plane using a Heaviside function ($H$) plus a screw dislocation in the opposite sense to the plate motion at depth $d_2$. We also retain the static offset, $c$, and include a new offset to the fault location, $x_c$. Sometimes the observed signal may not exactly align with the mapped fault trace, and so it can be useful to include a small offset to the location of the fault.

Let's look at the profile across a creeping part of the Xianshuihe Fault.

In [ ]:
# creep
prof_start = (830, 3410)
prof_end = (650, 3265)

prof_params = {
    "nbins": 100, # number of bins to split the profile into
    "width": 5  # total width of the profile in km
}

# run the profiler
bin_val, prof_bin_mids, points_val, points_dist, points_poly \
    = lib.profile_data(xx_utm,yy_utm,vel_para,prof_start,prof_end,prof_params)

# calculate profile-fault intersection angle
intersect_dist, intersect_angle = lib.profile_fault_intersection(prof_start,prof_end,fault_trace_utm)

# plot the profile in map view and in cross-section
lib.plot_profile(xx_utm, yy_utm, vel_para, fault_trace_utm, prof_start, prof_end, points_poly, xlim, ylim, 
                 points_dist, points_val, prof_bin_mids, bin_val, intersect_dist, intersect_angle) 


Next, we'll run a forward model.

In [ ]:
# Slip and locking depth
#####################################
s1 = 10 # mm/yr
s2 = 9 # mm/yr
d1 = 18 # km
d2 = 7 # km
c = -8 # mm/yr
xc = 0 # km
#####################################

# shift profile so that fault is at 0 km
x_prof = points_dist - intersect_dist

# Vector of positions -120 km to 120 km, sampling at every 100 m to better capture the creep motion near fault
x = np.arange(-120,121,0.1)

# run the forward model
v = lib.fault_creep(x*1000, s1/1000, s2/1000, d1*1000, d2*1000, c/1000, xc=xc*1000)

# calculate rms misfit by running the forward model for x positions with velocities
v_forward = lib.fault_creep(x_prof*1000, s1/1000, s2/1000, d1*1000, d2*1000, c/1000, xc=xc*1000)
rms_forward = lib.rms_misfit(points_val,v_forward*1000)

# plot profile with your model
lib.plot_creep_profile(x_prof, points_val, x, v, s1, s2, d1, d2, c, rms_forward)

What's your best answer? 

Let us know by filling out this form:

https://forms.gle/jPDE26v68BiN98bb6

## 7. Estimating shear strain rate from 1-D velocity profiles

Estimates of strain rate can highlight areas of concentrated deformation, an important product for understanding both seismic hazard and the wider-scale kinematics. The state of strain in the lithosphere can be described using a strain rate tensor, $\dot{\varepsilon}$:

\begin{equation}
    \dot{\varepsilon} = \begin{bmatrix}
    \frac{\partial V_E}{\partial x} & \frac{1}{2}\left(\frac{\partial V_E}{\partial y}+\frac{\partial V_N}{\partial x}\right) & \frac{1}{2}\left(\frac{\partial V_E}{\partial z}+\frac{\partial V_U}{\partial x}\right) \\    
    \frac{1}{2}\left(\frac{\partial V_N}{\partial x}+\frac{\partial V_E}{\partial y}\right) & \frac{\partial V_N}{\partial y} & \frac{1}{2}\left(\frac{\partial V_N}{\partial z}+\frac{\partial V_U}{\partial y}\right) \\    
    \frac{1}{2}\left(\frac{\partial V_U}{\partial x}+\frac{\partial V_E}{\partial z}\right) & \frac{1}{2}\left(\frac{\partial V_U}{\partial y}+\frac{\partial V_N}{\partial z}\right) & \frac{\partial V_U}{\partial z}    
    \end{bmatrix} \qquad \qquad (6)
\end{equation}

where $v_E$, $v_N$, and $v_U$ are the velocities in the Cartesian coordinate direction, $x$, $y$, and $z$.

In this instance, we have fault-parallel velocities at distances perpendicular to the fault, and so we can only estimate the shear strain across the fault. For our screw dislocation model, we can forward model the shear strain, $e_{shear}$, using (Savage & Burford, 1973):

\begin{equation}
e_{shear} = \left(\frac{sd}{2\pi}\right)\left(x^2+d^2\right)^{-1} \qquad \qquad (7)
\end{equation}

Let's start by calculating the shear rate for the screw dislocation model that we fit to our original profile, using both a forward model (Equation 7) and by calculating the gradient in the modelled velocities.

In [ ]:
# Slip rate and locking depth
#####################################
s = 5 # mm/yr
d = 1 # km
c = -8 # mm/yr
#####################################

# Original profile line
prof_start = (830, 3410)
prof_end = (650, 3265)

prof_params = {
    "nbins": 100, # number of bins to split the profile into
    "width": 5  # total width of the profile in km
}

# run the profiler
bin_val, prof_bin_mids, points_val, points_dist, points_poly \
    = lib.profile_data(xx_utm,yy_utm,vel_para,prof_start,prof_end,prof_params)

# calculate profile-fault intersection angle
intersect_dist, intersect_angle = lib.profile_fault_intersection(prof_start,prof_end,fault_trace_utm)

# shift profile so that the fault is at 0 km
x_prof = points_dist - intersect_dist

# Vector of positions every 1 km for -120 km to 120 km
x = np.arange(-120,121,1)

# run the forward model
v = lib.screw_disc(x*1000, s/1000, d*1000, c/1000)

# calculate shear strain rate using Equation 7
e_shear = lib.shear_strain(x*1000, s/1000, d*1000)

# also calculate shear strain rate based purely on velocity gradient
e_shear_grad = np.gradient(v,x*1000)

# plot the velocity and strain profile
lib.plot_strain_profile(x_prof, points_val, x, v, intersect_angle, e_shear, e_shear_grad)

The factor of two difference between the modelled strain rate and estimated strain rate reflects a difference between engineering shear strain and tensor strain rate. For further information, have a look at the Supporting Information of Weiss et al. (2020).

We can get nice, smooth estimates of strain rate from our screw dislocation models. In real studies, we often want to estimate strain rates directly from our velocities, as this will capture signals that are not represented in our model. However, we still require some degree of smoothing to suppress short-wavelength noise in the InSAR. This noise, while small compared to the tectonic signal, can cause large gradients between adjacent points that then drown out the underlying tectonic strain rate.

Let's plot both the strain rate directly calculated from the binned profiled fault-parallel velocities, and one calculated from the same velocities but smoothed with a sliding window average. Try changing the size of the smoothing window.

In [ ]:
# Window size for smoothing, must be an odd number
#####################################
wind_size = 5
#####################################

# shift fault location to 0 km
x_prof = prof_bin_mids - intersect_dist

# calculate gradient of original binned velocities
e_shear = np.gradient(bin_val/1000, x_prof*1000)

# apply a sliding mean window to smooth out some of the noise
x_smoothed, vel_smoothed = lib.sliding_window_mean(x_prof, bin_val, wind_size)

# calculated the gradient from the smoothed velocities
e_shear_smoothed = np.gradient(vel_smoothed/1000, x_smoothed*1000)

# Plot comparison
fig, axs = plt.subplots(1,1,figsize=(15,8))

plt.plot(x_prof, e_shear, c='r', label='Original binned velocities')
plt.plot(x_smoothed, e_shear_smoothed, c='b', label='Smoothed velocities')
plt.plot([0, 0],[axs.get_ylim()[0], axs.get_ylim()[1]], color='grey', linestyle='dashed')
plt.xlabel('Distance from fault (km)')
plt.ylabel('Shear strain rate (/yr)')
plt.legend(fontsize="x-large")

plt.show()

If we calculate the strain rate directly from the binned velocities (which have themselves already been smoothed to some degree), then the strain signal we would expect to see based on Equation 6 is completely masked.

By applying a smoothing window (a window size of 5 works quite well), we can bring out the higher strain across the fault trace. If the window size is increased too far, then we can actually smooth out the original interseismic signal, and this peak disappears again. How we filter out high-frequency noise in the velocities is integral to our estimates of strain rate.

For further reading, I recommend Weiss et al. (2020), who apply the Velmap method to all of Turkey, and Ou et al. (2022), who use a median filter method to localise strain onto a number of faults in the Tibetan Plateau.

## 8. Estimating strain rates from 2-D surface velocities.

In addition to estimating strain rates from profiled velocities, we can also calculate strain rate in 2-D from our decomposed fault-parallel velocities. Based on our previous assumption of zero fault-perpendicular motion, we can split the fault-parallel velocities into North and East components, allowing us to calculate the gradients of both in $x$ and $y$.

This method is based on the work of: <br>
*Ou, Q., Daout, S., Weiss, J. R., Shen, L., Lazecký, M., Wright, T. J., & Parsons, B. E. (2022). Large‐Scale Interseismic Strain Mapping of the NE Tibetan Plateau From Sentinel‐1 Interferometry. Journal of Geophysical Research: Solid Earth, 127(6), e2022JB024176.*

As with the profiled velocities, we'll first need to filter the decomposed fault-parallel velocities to remove any short-wavelength noise. Try experimenting with the filter type/width (https://www.pygmt.org/dev/api/generated/pygmt.grdfilter.html).

In [ ]:
# Set filter type (e.g., m for median; g for gaussian) and width (degrees) 
#####################################
filter = "m0.2"
#####################################

# Define working region
lon_min = 100 
lon_max = 102.5 
lat_min = 29.2 
lat_max = 31 
region = f"{lon_min}/{lon_max}/{lat_min}/{lat_max}" 
pixel_size = 0.01 # degrees 

# Convert from an ascii three-column list of xyz, to a binary grid 
vel_para_grd = pygmt.xyz2grd(x=xx_regrid.flatten(), y=yy_regrid.flatten(), z=vel_para.flatten(), region=region, spacing=pixel_size) 

# Apply filter to mitigate short-wavelength signals
vel_para_filt_grd = pygmt.grdfilter(grid=vel_para_grd, filter=filter, distance="0", region=region, spacing=pixel_size) 

# convert to numpy and flip array upside down to match default origin
vel_para_filt_array = np.flipud(vel_para_filt_grd.to_numpy())

# plot the filtered velocities
xlim = (lon_min, lon_max)
ylim = (lat_min, lat_max)
fig, ax = plt.subplots(figsize=(10,10))
im = ax.imshow(vel_para_filt_array, interpolation='none', cmap=cm.vik, 
               vmin=-20, vmax=20, extent=(lon_min,lon_max,lat_min,lat_max))
ax.plot(fault_trace[:,0], fault_trace[:,1], color="red")
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="2%", pad="2%")
plt.colorbar(im, cax=cax, label='Fault-parallel velocity (mm/yr)')
ax.set_aspect('equal', 'box')
ax.set_title('Filtered fault-parallel velocities')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_xlim(xlim)
ax.set_ylim(ylim)
plt.show()

In [ ]:
# calculate Ve and Vn from filtered fault-parallel velocities
vel_east = strike_E * vel_para_filt_array
vel_north = strike_N * vel_para_filt_array

# plot velocity components
fig, axs = plt.subplots(1,2, figsize=(17,5), sharey='all')

# East velocity
im = axs[0].imshow(vel_east, interpolation='none',cmap=cm.vik, 
                   vmin=-10, vmax=10, extent=(lon_min,lon_max,lat_min,lat_max))
axs[0].plot(fault_trace[:,0], fault_trace[:,1], color="red")
divider = make_axes_locatable(axs[0])
cax = divider.append_axes("right", size="2%", pad="2%")
plt.colorbar(im, cax=cax, label='Ve (mm/yr)')
axs[0].set_aspect('equal', 'box')
axs[0].set_title('East velocity')
axs[0].set_xlabel('Longitude')
axs[0].set_ylabel('Latitude')
axs[0].set_xlim(xlim)
axs[0].set_ylim(ylim)

# North velicity
im = axs[1].imshow(vel_north, interpolation='none',cmap=cm.vik, 
                   vmin=-10, vmax=10, extent=(lon_min,lon_max,lat_min,lat_max))
axs[1].plot(fault_trace[:,0], fault_trace[:,1], color="red")
divider = make_axes_locatable(axs[1])
cax = divider.append_axes("right", size="2%", pad="2%")
plt.colorbar(im, cax=cax, label='Vn (mm/yr)')
axs[1].set_aspect('equal', 'box')
axs[1].set_title('North velocity')
axs[1].set_xlabel('Longitude')
axs[1].set_xlim(xlim)
axs[1].set_ylim(ylim)

plt.show()

In [ ]:
pixel_size = 1  # km 
# calculate velocity gradients
dVedy = (vel_east[1:]-vel_east[:-1]) / pixel_size / 10**6  # mm/yr velocity difference divided by pixel spacing in km
dVedx = (vel_east[:,1:]-vel_east[:,:-1]) / pixel_size / 10**6  
dVndy = (vel_north[1:]-vel_north[:-1]) / pixel_size / 10**6  # mm/yr velocity difference divided by pixel spacing in km
dVndx = (vel_north[:,1:]-vel_north[:,:-1]) / pixel_size / 10**6  

# # calculate shape of final strain rate field, 1 pixel smaller than velocity grid because of gradients
strain_shape0 = vel_east.shape[0]-1
strain_shape1 = vel_east.shape[1]-1

# calculate elements of a 2x2 horizontal strain tensor 
exx = dVedx[:strain_shape0, :strain_shape1]
eyy = dVndy[:strain_shape0, :strain_shape1]
exy = (dVedy[:strain_shape0, :strain_shape1]+dVndx[:strain_shape0, :strain_shape1])/2

# calculate maximum shear 
shear = np.sqrt(exy**2 + ((exx-eyy)**2)/4) # in units of strain
shear = shear * 10**9 # in units of nanostrain

# mask strains where velocities are NaN 
vel_crop = vel_para_grd[:-1, :-1] # velocities are 1 pixel larger 
shear_masked = np.where(np.isnan(vel_crop), np.nan, shear) 

# plot maximum shear
fig, ax = plt.subplots(figsize=(10,10))
im = ax.imshow(shear_masked, interpolation='none',cmap=cm.lajolla_r, vmin=0, vmax=500, extent=(lon_min,lon_max,lat_min,lat_max))
ax.plot(fault_trace[:,0], fault_trace[:,1], color="red")
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="2%", pad="2%")
plt.colorbar(im, cax=cax, label='Shear (nst/yr)')
ax.set_aspect('equal', 'box')
ax.set_title('Maximum Shear')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_xlim(xlim)
ax.set_ylim(ylim)
plt.show()

## 9. Extension - Bayesian inversion for profile modelling

Bayesian inversions are one way we can estimate model parameters and their uncertainties in geophysics. We take some prior knowledge about the model (e.g. limits of the model parameter values) and combine this with a likelihood function based on the observations to estimate the posterior probability (i.e. the likelihood of our model parameters being the true values given the data). By exploring a range of model parameter values, we can build up a posterior probability distribution which tells us both our "best fit" values, known as the "maximum a posteriori probability (MAP)" solution, and allows us to estimate the uncertainties on this solution.

We'll assume uniform priors for each model parameter, meaning all values between the upper and lower limits are equally likely. Values outside of these limits are "impossible" and will be rejected in our inversion.

We use the following limits for our uniform priors:
- $0 \leq s \leq 40$ mm/yr
- $1 \leq d \leq 50$ km
- $-10 \leq c \leq 10$ mm/yr

Assuming our errors are Gaussian, we can find the maximum likelihood by minimising the weighted difference between the observed velocities, $\mathbf d$, and the model velocities, $g(\mathbf{m})$:
\begin{equation}
    \sum (\mathbf d-g(\mathbf{m}))*W*(\mathbf d-g(\mathbf{m}))
\end{equation}
where $W$ contains the weight for every velocity. Weighting the data is useful when some velocities have larger errors than others. For simplicity, we'll assume a uniform uncertainty of 1 mm/yr (1 standard deviation) across all of our velocities.

In order to explore the posterior distribution (i.e. test a large number of model parameter values), we'll use a Markov Chain Monte Carlo (MCMC) method.
Our MCMC performs the following steps:
- Take a starting model and add a random step to each parameter.
- Test whether this new model is within the prior limits, rejecting the model if not.
- Calculate the likelihoods for both models.
- If the new model has a higher likelihood, then accept this model. Also accept a number of models that are worse to help the Monte Carlo explore the full parameter space.

We need to allow the MCMC to sometimes accept less likely models to avoid local minima. These are models that are good compared to the surrounding models (in the parameter space), but are not the true best model (called the global minima).

In [ ]:
importlib.reload(lib)
np.random.seed(1)

# inversion setup
#####################################
n_iterations = 10000 # don't go above this value of 10,000 in the practical
#####################################

# take profiled velocities and shift the fault to be at zero
x_prof = points_dist - intersect_dist
v_prof = points_val

# model params and their limits
m_start = np.array([5, 5, 0], dtype=np.double) # slip in mm/yr, locking depth in km
m_min = np.array([0, 1, -10], dtype=np.double)
m_max = np.array([40, 50, 10], dtype=np.double)

# variance-covariance matrix, currently 1 mm/yr uncertainties
vcm = np.eye(len(x_prof))

# convert all units to metres
x = x_prof * 1000
v = v_prof / 1000
m_start = m_start * np.array([0.001, 1000, 0.001])
m_min = m_min * np.array([0.001, 1000, 0.001])
m_max = m_max * np.array([0.001, 1000, 0.001])
vcm = vcm / 1e6

# inversion setup
burn_in = round(n_iterations/100)
acceptance_coef = 0.001

# weightings
W = np.linalg.inv(vcm)

# pre-allocate arrays
models_saved=np.zeros((n_iterations,len(m_start))) * np.nan
ll_saved=np.zeros((n_iterations,1)) * np.nan
n_accept = 0 # number of accepted models
n_reject = 0 # number of rejected models

# pass starting model to current model
m_current = m_start.copy()

# run inversion
for ii in range(n_iterations):
    
    # propose model using different step sizes for each parameter
    m_trial = m_current.copy()
    m_trial[0] = m_trial[0] + np.random.normal(loc=0, scale=2.5, size=1)/1000 # slip rate
    m_trial[1] = m_trial[1] + np.random.normal(loc=0, scale=2.5, size=1)*1000 # locking depth
    m_trial[2] = m_trial[2] + np.random.normal(loc=0, scale=1, size=1)/1000 # offset
    
    # check limits and skip the rest of the loop if any parameter is invalid
    if not(lib.prior(m_trial,m_min,m_max)):
        n_reject += 1
        models_saved[ii,:] = m_current
        continue
    
    # calculate likelihood for the current and trial models
    ll_current = lib.likelihood(x, v, m_current, W)
    ll_trial = lib.likelihood(x, v, m_trial, W)
    
    # test whether to keep trial model
    #if np.exp(acceptance_coef*(ll_trial-ll_current)) > np.random.uniform(low=0, high=1,size=1):
    if np.exp(acceptance_coef*(ll_current-ll_trial)) > np.random.uniform(low=0, high=1,size=1):
        m_current = m_trial
        ll_current = ll_trial
        n_accept += 1
    else:
        n_reject += 1
    models_saved[ii,:] = m_current
    ll_saved[ii] = ll_current
    
# convert back from metres to mm/yr and km
models_saved[:,0] = models_saved[:,0] * 1000 # slip rate
models_saved[:,1] = models_saved[:,1] / 1000 # locking depth
models_saved[:,2] = models_saved[:,2] * 1000 # offset

# find best fit model using min of likelihood function
best_model = models_saved[np.nanargmin(ll_saved),:]

print('Inversion complete')
print('Number of accepted models = ' + str(n_accept))
print('Number of rejected models = ' + str(n_reject))

First, we'll plot the accepted trial models for the slip rate and locking depth to see how the algorithm explores the parameter space.
We'll show the first 1% of iterations in red and the rest in blue.
These early iterations are often termed a "burn-in" period, where the inversion is moving from the start position towards the (global) minimum, which it then begins to explore.

In [ ]:
# plot walker 
plt.figure()
plt.plot(models_saved[:burn_in,0], models_saved[:burn_in,1], color='red', alpha=0.3)
plt.plot(models_saved[burn_in:,0], models_saved[burn_in:,1], color='blue', alpha=0.3)
plt.xlabel('Slip rate (mm/yr)')
plt.ylabel('Locking depth (km)')
plt.show()

We can also look at how the inversion explored the parameter space by plotting the likelihood function value for each iteration. The lowest likelihood will be located within a minima, while the higher likelihoods throughout show the inversion accepting worse models to help it escape local minima.

In [ ]:
# plot likelihood value throughout Monte Carlo
fig, axs = plt.subplots(1,1,figsize=(15,8))

axs.scatter(np.arange(1,len(ll_saved)+1),ll_saved, s=3)
axs.set_xlabel('Number of iterations')
axs.set_ylabel('Weighted residual')

plt.show()

Next, let's plot our best model on top of the profiled velocities.

In [ ]:
# calculate rms misfit
v_final = lib.screw_disc(x_prof, best_model[0], best_model[1], best_model[2])
best_rms = lib.rms_misfit(v_prof,v_final)

# Discard burn-in 
v_all_postburn = np.zeros((n_iterations - burn_in, len(x))) 

for ii in range(burn_in, n_iterations):
    v_all_postburn[ii - burn_in, :] = lib.screw_disc(
        x,
        models_saved[ii, 0],  # slip
        models_saved[ii, 1],  # locking depth
        models_saved[ii, 2]   # offset
    )

# Compute 95% confidence interval
v_lower = np.percentile(v_all_postburn, 2.5, axis=0)
v_upper = np.percentile(v_all_postburn, 97.5, axis=0)
# for plotting
x = np.arange(-100,101,1)
v_final_plot = lib.screw_disc(x, best_model[0], best_model[1], best_model[2])

lib.plot_profile_model(x_prof, v_prof, x, v_final_plot/1000, round(best_model[0],2), round(best_model[1],2), round(best_model[2],2), best_rms)


Now we'll plot histograms for each model parameter. The overall shape of these histograms should be Gaussian, although they may be skewed to one side. They may also be quite rough depending on the number of iterations that have been run. By fitting a Gaussian to each histogram, we can estimate the standard deviation of each, which is a measure of the uncertainty on our estimates.

We'll plot the best model values as vertical red lines. These should roughly line up with the peaks of our histograms.

In [ ]:
# calculate mean and standard deviation for each parameter
gauss_s = stats.norm.fit(models_saved[burn_in:,0])
gauss_d = stats.norm.fit(models_saved[burn_in:,1])
gauss_c = stats.norm.fit(models_saved[burn_in:,2])

# create figure
fig, axs = plt.subplots(1, 3, figsize=(15,8))

# plot histograms
_, bins_s, _ = axs[0].hist(models_saved[burn_in:,0], bins=20, density=True)
_, bins_d, _ = axs[1].hist(models_saved[burn_in:,1], bins=20, density=True)
_, bins_c, _ = axs[2].hist(models_saved[burn_in:,2], bins=20, density=True)

# create gaussian fit lines
gauss_fit_s = stats.norm.pdf(bins_s, gauss_s[0], gauss_s[1])
gauss_fit_d = stats.norm.pdf(bins_d, gauss_d[0], gauss_d[1])
gauss_fit_c = stats.norm.pdf(bins_c, gauss_c[0], gauss_c[1])

# plot gaussian fits
axs[0].plot(bins_s, gauss_fit_s)
axs[1].plot(bins_d, gauss_fit_d)
axs[2].plot(bins_c, gauss_fit_c)

# label mean and sd
axs[0].text(0.02, 0.97, 'mean = ' + str(round(gauss_s[0],2)), fontsize=14, transform = axs[0].transAxes)
axs[0].text(0.02, 0.94, 'sd = ' + str(round(gauss_s[1],2)), fontsize=14, transform = axs[0].transAxes)
axs[1].text(0.02, 0.97, 'mean = ' + str(round(gauss_d[0],2)), fontsize=14, transform = axs[1].transAxes)
axs[1].text(0.02, 0.94, 'sd = ' + str(round(gauss_d[1],2)), fontsize=14, transform = axs[1].transAxes)
axs[2].text(0.02, 0.97, 'mean = ' + str(round(gauss_c[0],2)), fontsize=14, transform = axs[2].transAxes)
axs[2].text(0.02, 0.94, 'sd = ' + str(round(gauss_c[1],2)), fontsize=14, transform = axs[2].transAxes)

# plot best model values
axs[0].plot([best_model[0], best_model[0]],[0, axs[0].get_ylim()[1]], color="red")
axs[1].plot([best_model[1], best_model[1]],[0, axs[1].get_ylim()[1]], color="red")
axs[2].plot([best_model[2], best_model[2]],[0, axs[2].get_ylim()[1]], color="red")

axs[0].set_title('Slip rate', fontsize=16)
axs[1].set_title('Locking depth', fontsize=16)
axs[2].set_title('Offset', fontsize=16)

axs[0].set_xlabel('Slip rate (mm/yr)', fontsize=14)
axs[1].set_xlabel('Locking depth (km)', fontsize=14)
axs[2].set_xlabel('Offset (mm/yr)', fontsize=14)

fig.tight_layout()
plt.show()

As we saw in the "walker" plot, there is a trade-off between slip rate and locking depth, with an increase in one producing an increase in the other.

Let's replot the histograms above while also show the distribution of each model parameter against the other. This should make correlations between model parameters clearer to see. We'll use the Seaborn package for this, as it has a pre-made function for generating these multivariate plots. It's well worth an investigation if you want to use Python to plot more scientific data in the future.

In [ ]:
# This plot can take a couple of minutes to run (dependent on the number of iterations)

# multivariate plot
models_df = pd.DataFrame(data=models_saved, columns=["Slip rate (mm/yr)", "Locking depth (km)", "Offset (mm/yr)"])
sns.set_style("white") 
g = sns.PairGrid(models_df, corner=True, diag_sharey=False)
g.map_diag(sns.histplot, kde=True, color="skyblue")
g.map_lower(sns.kdeplot, fill=True, levels=10, thresh=0.1, color="steelblue") # use this to plot as contours instead (runs slower)

# add red dots for our best model
g.axes[1,0].scatter(best_model[0], best_model[1], c='red', s=50, edgecolor='black')
g.axes[2,0].scatter(best_model[0], best_model[2], c='red', s=50, edgecolor='black')
g.axes[2,1].scatter(best_model[1], best_model[2], c='red', s=50, edgecolor='black')

g.fig.set_size_inches(8,8)
plt.tight_layout() 
plt.show()

In [ ]:
# compare just slip rate and locking depth
models_df = pd.DataFrame(data=models_saved[:,0:2], columns=["Slip rate (mm/yr)", "Locking depth (km)"])
ax = sns.jointplot(data=models_df, x="Slip rate (mm/yr)", y="Locking depth (km)", kind="hex", height=8)
plt.show()

## 10. Extension - inverting geodetic strain rates for fault slip rates 

Geodetic datasets are widely used to estimate fault slip rates, and an increasing number of modeling approaches have been developed for this purpose. In this practical, we demonstrated how to derive slip rates by modeling profiles; however, this approach becomes impractical in regions with dense fault networks. 

Block modeling, which divides a region into small blocks that rotate relative to each other, can be efficient when geodetic data are sparse. However, it becomes challenging, particularly in areas lacking mapped faults or with complex arrays of active fault structures. In this method, all strains must occur along block boundaries, and every block must be fully enclosed by faults — even when some of these faults may not actually exist. 

Both profile and block modeling approaches rely on geodetic velocity fields, which require considering rotation. 

An alternative is to invert geodetic strain rate fields directly for fault slip deficit rates. A key advantage of this method is that it avoids the need to account for rigid block rotations, so tectonic blocks or microplates do not need to be defined. In addition to estimating fault slip rates, this approach can also resolve off-fault distributed moment sources that explain the observed diffuse deformation (Figure 6). More details can be found in Johnson et al. (2024) and Fang et al. (2024). 

<img src="figures/inverting_strain_rates.png" style="float: centre;" width="800"/>

*Figure 6: Inverting strain rates for fault slip rates and distributed moment sources ([Fang et al., 2024](https://doi.org/10.1029/2024GL111199)).*

## Conclusion

Hopefully, you now have a basic understanding of how we measure and model interseismic deformation. This practical has been shared under an open-access licence (see repo), so feel free to pass it on and utilise any of the underlying code. 

If you are interested in performing velocity decompositions on a larger scale, have a look at our open-access Matlab package:
https://github.com/andwatson/decompose_insar_velocities.

If you would like to reference InSAR LOS velocities to 3D GNSS using the method in Ou et al. (2022), check out the Python code deposited on Zenodo: 
https://zenodo.org/record/6546922.

An alternative approach for referencing InSAR LOS velocities to 3D GNSS is to use VELMAP (Wang & Wright, 2012), available at: https://github.com/nerc-comet/velmap.

If you are interested in a more developed version of the Bayesian inversion, have a look at both Hussain et al. (2016) and the GBIS inversion software, for interseismic and coseismic deformation, respectively:
https://comet.nerc.ac.uk/gbis/

If you are interested in the direct inversion of strain rates for slip rates, check out Fang et al. (2024) and the accompanying code archived on Zenodo: 
https://doi.org/10.5281/zenodo.12583880.

If you found any issues in this practical, please log them under "issues" on the github repository.

Presenter: [Jin Fang](https://environment.leeds.ac.uk/see/staff/13057/dr-jin-fang) (E-mail: J.Fang@leeds.ac.uk), University of Leeds

### References

Elliott, J. (2009). Strain accumulation & release on the Tibetan Plateau measured using InSAR (Doctoral dissertation, Oxford University).

Fang, J., Wright, T. J., Johnson, K. M., Ou, Q., Styron, R., Craig, T. J., ... & Zheng, G. (2024). Strain partitioning in the Southeastern Tibetan Plateau from kinematic modeling of high‐resolution Sentinel‐1 InSAR and GNSS. Geophysical Research Letters, 51(19), e2024GL111199.

Hussain, E., Wright, T. J., Walters, R. J., Bekaert, D., Hooper, A., & Houseman, G. A. (2016). Geodetic observations of postseismic creep in the decade after the 1999 Izmit earthquake, Turkey: Implications for a shallow slip deficit. Journal of Geophysical Research: Solid Earth, 121(4), 2980-3001.

Johnson, K. M., Wallace, L. M., Maurer, J., Hamling, I., Williams, C., Rollins, C., ... & Van Dissen, R. (2024). Inverting geodetic strain rates for slip deficit rate in complex deforming zones: An application to the New Zealand plate boundary. Journal of Geophysical Research: Solid Earth, 129(3), e2023JB027565. 

Watson, A. R., Elliott, J. R., & Walters, R. J. (2022). Interseismic strain accumulation across the Main Recent Fault, SW Iran, from Sentinel-1 InSAR observations. Journal of Geophysical Research: Solid Earth, 127, e2021JB022674. 

Ou, Q., Daout, S., Weiss, J. R., Shen, L., Lazecký, M., Wright, T. J., & Parsons, B. E. (2022). Large‐Scale Interseismic Strain Mapping of the NE Tibetan Plateau From Sentinel‐1 Interferometry. Journal of Geophysical Research: Solid Earth, 127(6), e2022JB024176.

Savage, J. C., & Burford, R. O. (1973). Geodetic determination of relative plate motion in central California. Journal of Geophysical Research, 78(5), 832-845.

Vernant, P. (2015). What can we learn from 20 years of interseismic GPS measurements across strike-slip faults?. Tectonophysics, 644, 22-39.

Weiss, J. R., et al. (2020). High‐resolution surface velocities and strain for Anatolia from Sentinel‐1 InSAR and GNSS data. Geophysical Research Letters, 47(17), e2020GL087376.

Wang, D., Elliott, J. R., Zheng, G., Wright, T. J., Watson, A. R., McGrath, J. D. (2024). Deciphering interseismic strain accumulation and its termination on the central-eastern Altyn Tagh fault from high-resolution velocity fields. Earth and Planetary Science Letters, 644, 118919.